In [1]:
from lxml import etree as ET
import requests, zipfile, io
import pandas as pd
import os

In [2]:
tree = ET.parse('python_assesment.xml')
root = tree.getroot()
#root = ET.fromstring(python_assesment_as_string)

In [3]:
dlink = None
path_name = os.path.abspath(os.getcwd())
path_name += "\DLTINS"

In [4]:
for doc in root.iter('doc'):
    for child in doc:
        if child.get('name') == 'download_link':
            dlink = child.text
        if child.get('name') == 'file_type' and child.text != 'DLTINS':
            dlink = None
    r = requests.get(dlink)
    z = zipfile.ZipFile(io.BytesIO(r.content))
    z.extractall(path=path_name)

In [5]:
cnt = 1
id, fullnm, clssfctnTP, cmmdtyDeriveInd, ntnlCcy, issr = None, None, None, None, None, None
for xmlfile in os.listdir(path_name):
    if xmlfile.endswith(".xml"):
        root = ET.parse(path_name + "/" + xmlfile)

        for elem in root.getiterator():
            elem.tag = ET.QName(elem).localname
        ET.cleanup_namespaces(root)
        
        cols = ["FinInstrmGnlAttrbts.Id",
                "FinInstrmGnlAttrbts.FullNm", 
                "FinInstrmGnlAttrbts.ClssfctnTp", 
                "FinInstrmGnlAttrbts.CmmdtyDerivInd", 
                "FinInstrmGnlAttrbts.NtnlCcy", 
                "Issr"]
        rows = []
        
        for finstrm in root.iter('FinInstrm'):
            #trmntrecord = finstrm.find('TermntdRcrd')
            for finattrb in finstrm.iter('FinInstrmGnlAttrbts'):
                id = finattrb.find("Id").text
                fullnm = finattrb.find("FullNm").text
                clssfctnTP = finattrb.find("ClssfctnTp").text
                cmmdtyDeriveInd = finattrb.find("CmmdtyDerivInd").text
                ntnlCcy = finattrb.find("NtnlCcy").text
                issr = finattrb.find("./../Issr").text
                #print(cnt, id, fullnm, clssfctnTP, cmmdtyDeriveInd, ntnlCcy, issr)

                rows.append({"FinInstrmGnlAttrbts.Id": id,
                             "FinInstrmGnlAttrbts.FullNm": fullnm,
                             "FinInstrmGnlAttrbts.ClssfctnTp": clssfctnTP,
                             "FinInstrmGnlAttrbts.CmmdtyDerivInd": cmmdtyDeriveInd,
                             "FinInstrmGnlAttrbts.NtnlCcy": ntnlCcy,
                             "Issr": issr})
        #print(id, fullnm, clssfctnTP, cmmdtyDeriveInd, ntnlCcy, issr)
    
        df = pd.DataFrame(rows, columns=cols)
        df.to_csv(path_name + "/output" + str(cnt) + ".csv")
        cnt += 1